In [110]:
import warnings
import asyncio
import nest_asyncio

import yfinance as yf

from IB_strats import *

%load_ext autoreload
%autoreload 2

warnings.filterwarnings('ignore', category=FutureWarning)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [111]:
async def run_strategy():
    ib = IB()
    await ib.connectAsync('127.0.0.1', 7497, clientId=1)
    try:
        ticker = 'ETH-USD'
        start_date = datetime(2020, 1, 1)
        end_date = date.today() - timedelta(days=1)
        eth_data = dati_giornalieri(ticker, start_date, end_date)
        capital = 100
        asset = 'ETH'
        trade_signals_aaronx = signals_aaron_x(eth_data, asset, capital)
        trade_signals_aarony = signals_aaron_y(eth_data, asset, capital)
        trade_signals_obv = signals_obv(eth_data, asset, capital)
        trade_signals_kama = signals_kama(eth_data, asset, capital)
        trade_signals_stoch = signals_stoch(eth_data, asset, capital)
        trade_signals_psar = signals_psar(eth_data, asset, capital)
        execute_trades_aaron_x(ib, trade_signals_aaronx)
        execute_trades_aaron_y(ib, trade_signals_aarony)
        execute_trades_obv(ib, trade_signals_obv)
        execute_trades_kama(ib, trade_signals_kama)
        execute_trades_stoch(ib, trade_signals_stoch)
        execute_trades_psar(ib, trade_signals_psar)
    except Exception as e:
        print(f"Error in run_strategy: {e}")
    finally:
        ib.disconnect()

In [112]:
await run_strategy()

[*********************100%***********************]  1 of 1 completed
AaronX Strategy
AaronY Strategy
OBV Strategy
KAMA Strategy
Stochastic Strategy
SAR Strategy


In [113]:
async def schedule_daily_run():
    market_open_time = datetime.now().replace(hour=9, minute=30)
    while True:
        current_time = datetime.now()
        if current_time < market_open_time:
            sleep_seconds = (market_open_time - current_time).total_seconds()
            await asyncio.sleep(sleep_seconds)
            await run_strategy()
        await asyncio.sleep(24*60*60)

In [ ]:
await schedule_daily_run()